In [19]:
import numpy as np
import tensorflow as tf
images = np.load('trnImage.npy')
labels = np.load('trnLabel_coarse.npy')
#labels_fine = np.load('trnLabel_fine.npy')

test_images = np.load('tstImage.npy')
test_labels = np.load('tstLabel_coarse.npy')
#test_labels_fine = np.load('tstLabel_fine.npy')

print(f'Images Shape: {images.shape}')
print(f'Train Labels Coarse Shape: {labels.shape}')

Images Shape: (32, 32, 3, 50000)
Train Labels Coarse Shape: (50000,)


In [20]:
train_images_reshaped = np.transpose(images, (3, 0, 1, 2))
test_images_reshaped = np.transpose(test_images, (3, 0, 1, 2))
labels_reshaped = labels.reshape(50000, 1)
test_labels_reshaped = test_labels.reshape(10000, 1)
#labels_fine_reshaped = labels_fine.reshape(50000, 1)

x_train_normalised = np.divide(train_images_reshaped, 255)
x_test_normalised = np.divide(test_images_reshaped, 255)

x_train = x_train_normalised
y_train = labels_reshaped
#y_train = labels_fine_reshaped

x_test = x_test_normalised
y_test = test_labels_reshaped


#Train-validation-test split
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.3)


#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical



#Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)


# (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

# x_train, x_test = x_train / 255, x_test / 255 # normalize data

# print((x_train.shape,y_train.shape))
# print((x_test.shape,y_test.shape))



x_val = x_test
y_val = y_test

#Defining the parameters
batch_size= 25
epochs=2
learn_rate=.01

((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))


In [26]:
#Defining the parameters
batch_size= 50
epochs=1
learn_rate=.001

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.datasets import cifar100
import tensorflow as tf
import datetime

x_test = x_val
y_test = y_val

model = ResNet50(input_shape=x_train.shape[1:], weights=None, pooling ="max")
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'])

model.fit(
    train_generator.flow(x_train, y_train, batch_size=batch_size),
    epochs = epochs,
    steps_per_epoch = x_train.shape[0]//batch_size,
    validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size),
    validation_steps = len(x_test)//batch_size, verbose=1
)

 50/700 [=>............................] - ETA: 14:18 - loss: 4.6824 - accuracy: 0.0427